In [1]:
import findspark

findspark.init("/home/ozan/spark")

from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder\
.config("spark.jars","/home/ozan/mysql-connector-java-8.0.27.jar")\
.master("local[4]")\
.appName("mysql")\
.getOrCreate()                                                     

In [4]:
# builder: spark sesionu kurmamıza sağlıyor.
# config: jar dosyasının yerini belirtiyoruz.
# master(local[4]) sparkın çalıştığı kümeyi almızı sağlar burada local makinede 4 bölümde çalıştırdığımızı gösteriyor.
# appName("beginner") uygulama adı.
# getOrCreate() varsa spark nesnesi döndürür yoksa yenisini oluşturur.

In [6]:
mysqlDF=spark.read\
.format("jdbc")\
.option("driver","com.mysql.jdbc.Driver")\
.option("url","jdbc:mysql://127.0.0.1:3306/bike")\
.option("dbtable","stations")\
.option("user","root")\
.option("password","1")\
.load()                                           

In [7]:
#format: Veri formatı klasik veri tabanı olduğu için jdbc olarak giriyoruz.
#option: Gerekli parametreleri giriyoruz.
#driver: Bu parametre ile gerekli url'ye bağlanmak için jdbc sürücüsünün adı.
#url: Bu parametre ile veri tabanı url'sine bağlanıyoruz.
#dbtable : Tablonun adı.
#user : Kullanıcı adı.
#password : kullanıcı şifresi.

In [8]:
mysqlDF.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- capacity: integer (nullable = true)
 |-- hometown_id: string (nullable = true)



In [10]:
mysqlDF.show()

print("Toplam kayıt sayısı :"+str(mysqlDF.count()))

+----------+--------+-----------+
|station_id|capacity|hometown_id|
+----------+--------+-----------+
|      1001|      20|       2001|
|      1002|      20|       2001|
|      1003|      20|       2001|
|      1004|      20|       2002|
|      1005|      20|       2002|
|      1006|      20|       2003|
|      1007|      20|       2001|
|      1008|      20|       2002|
|      1009|      20|       2003|
+----------+--------+-----------+

Toplam kayıt sayısı :9


In [11]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

schema=StructType([StructField("hometown_id",StringType()),
                   StructField("name",StringType()),
                   StructField("city",StringType())    
])

csvDF=spark\
.read\
.option("encoding","ISO-8859-9")\
.option("header",True)\
.schema(schema)\
.csv("hometown.csv")

csvDF.show()

+-----------+-------+------+
|hometown_id|   name|  city|
+-----------+-------+------+
|       2001|Donanma|Gölcük|
|       2002| Merkez| izmit|
|       2003|Saraylı|Gölcük|
+-----------+-------+------+



In [12]:
newDF=mysqlDF.join(csvDF,csvDF.hometown_id==mysqlDF.hometown_id)
newDF.select(newDF.station_id,newDF.capacity,newDF.name,newDF.city).show()

+----------+--------+-------+------+
|station_id|capacity|   name|  city|
+----------+--------+-------+------+
|      1001|      20|Donanma|Gölcük|
|      1002|      20|Donanma|Gölcük|
|      1003|      20|Donanma|Gölcük|
|      1004|      20| Merkez| izmit|
|      1005|      20| Merkez| izmit|
|      1006|      20|Saraylı|Gölcük|
|      1007|      20|Donanma|Gölcük|
|      1008|      20| Merkez| izmit|
|      1009|      20|Saraylı|Gölcük|
+----------+--------+-------+------+

